In [11]:
import numpy as np
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
df=pd.read_csv('fake_job_postings.csv')

In [13]:
#import torch 
#torch.cuda.get_device_name(0)

In [14]:
#--------1. split the columns to country, state, and city

# to use country only?
df['country']=df.location.str.split(',',expand=True)[:][0]

df['state']=df.location.str.split(',',expand=True)[:][1]
df['city']=df.location.str.split(',',expand=True)[:][2]

In [15]:

#----------2. replacing different kinds of missing value to np.nan
df.state = df.state.str.strip()
df.state.fillna(value=np.nan, inplace=True)
df.state.replace('', np.nan, inplace=True)
df.state.replace(' ', np.nan, inplace=True)


#df['state'].sort_values().unique()

In [16]:
#----------3. clear the whitespaces and signs at the start/end

df.city = df.city.str.strip(' /:\\')

#---------4. replacing different kinds of missing value to np.nan
df.city.fillna(value=np.nan, inplace=True)
df.city.replace('', np.nan, inplace=True)
df.city.replace(' ', np.nan, inplace=True)
df['city']=df.city.str.lower()
#------

#print(df['city'].sort_values().unique().tolist())

In [17]:
df.salary_range.fillna(value=np.nan, inplace=True)
df.salary_range.replace('', np.nan, inplace=True)
df.salary_range.replace(' ', np.nan, inplace=True)

#---------4. spilt salary range into min and max

df['min_salary']=df.salary_range.str.split('-',expand=True)[:][0]
df['max_salary']=df.salary_range.str.split('-',expand=True)[:][1]

df.max_salary.fillna(value=np.nan, inplace=True)

In [18]:
#----------5. for entry of salary_range as date, max and min salary are grouped as null value
df.loc[df['max_salary'].isin(['Apr', 'Dec', 'Jun', 'Nov', 'Oct', 'Sep']),['max_salary', 'min_salary']]=np.nan
df.loc[df['min_salary'].isin(['Dec', 'Jun', 'Oct']),['max_salary', 'min_salary']]=np.nan

#convert them into numerical value
df[['min_salary','max_salary']] = df[['min_salary','max_salary']].astype(float)

#for regression model, need to impute NaN values to median/mean
# df['max_salary'].fillna(value=df['max_salary'].mean(), inplace=True)
# df['min_salary'].fillna(value=df['min_salary'].mean(), inplace=True)

In [19]:
#-------------------end of processing for column salary_range and location

In [20]:
#-------------------start cleaning for text columns

In [22]:
df_drop=df.drop(columns=['location', 'salary_range'])

In [23]:
df_drop

,job_id,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,...,required_experience,required_education,industry,function,fraudulent,country,state,city,min_salary,max_salary
0,1,Marketing Intern,Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,...,Internship,NaN,NaN,Marketing,0,US,NY,new york,NaN,NaN
1,2,Customer Service - Cloud Video Production,Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,...,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,NZ,NaN,auckland,NaN,NaN
2,3,Commissioning Machinery Assistant (CMA),NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,...,NaN,NaN,NaN,NaN,0,US,IA,wever,NaN,NaN
3,4,Account Executive - Washington DC,Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,...,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,washington,NaN,NaN
4,5,Bill Review Manager,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,...,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,fort worth,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,Sales,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,...,Mid-Senior level,NaN,Computer Software,Sales,0,CA,ON,toronto,NaN,NaN
17876,17877,Payroll Accountant,Accounting,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,...,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0,US,PA,philadelphia,NaN,NaN
17877,17878,Project Cost Control Staff Engineer - Cost Con...,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,...,NaN,NaN,NaN,NaN,0,US,TX,houston,NaN,NaN
17878,17879,Graphic Designer,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,...,Not Applicable,Professional,Graphic Design,Design,0,NG,LA,lagos,NaN,NaN


In [25]:
# this function for preprocessing text is used linked from the next function

def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'(?u)\b\w\w+\b')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords



In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/minghaooo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
df['text'] = df.apply(lambda row: (str(row['title']) + ' ' + 
                                   str(row['company_profile']) + ' ' + 
                                   str(row['description']) + ' ' + 
                                   str(row['requirements']) + ' ' + 
                                   str(row['benefits'])), axis = 1)

# Fill empty columns with "Unspecified"
df['text'] = df['text'].fillna("Unspecified")
    
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text, ngram_range = (1,2),   #TRY ADJUSTING ngram_range and min_df, max_df
                                min_df= 0.01, max_df= 0.6)

# Fit to the data and transform to feature matrix
text_column = vectoriser.fit_transform(df['text'])

# Convert sparse matrix to dataframe
text_column = pd.DataFrame.sparse.from_spmatrix(text_column)

# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}

# Rename each column using the mapping
for col in text_column.columns:
    text_column.rename(columns={col: col_map[col]}, inplace=True)
    

In [28]:
from sklearn import datasets, feature_extraction, decomposition
n, m = text_column.shape
k = 10
tSVD = decomposition.TruncatedSVD(n_components = k, random_state = 2022)
xtr = tSVD.fit_transform(text_column)


In [29]:
tSVD.feature_names_in_

array(['00', '000', '10', ..., 'young', 'zealand', 'zone'], dtype=object)

In [30]:
features = tSVD.feature_names_in_
for i, comp in enumerate(tSVD.components_):
  termsincomp = zip(features, comp)
  sorteditem = sorted(termsincomp, key = lambda x: x[1], reverse = True)[:10]
  print("Concept %d:") 
  for item in sorteditem:
    print(item[0])


Concept %d:
service
sales
customer
market
business
job
design
project
development
management
Concept %d:
abroad
job
get
celta
tefl
tesol
passport
teacher
teachers
kid
Concept %d:
customer
mail
service
document
productivity
perform
process
file
increase
communications
Concept %d:
apprenticeship
job
website
olds
wish
16
career
18
course
education
Concept %d:
website
job
engineer
manufacture
search
technical
redirect
apply
1500
click
Concept %d:
design
web
apprenticeship
mail
olds
developer
software
16
course
wish
Concept %d:
market
sales
media
website
digital
social
search
brand
campaign
content
Concept %d:
sales
project
business
management
development
account
client
data
solutions
software
Concept %d:
shall
media
digital
social
content
market
conflict
campaign
video
agency
Concept %d:
finance
employment
financial
credit
account
holiday
months
vacation
disability
status


In [37]:
text_column

,00,000,10,100,1000,11,12,13,14,15,...,writtenlifting,xml,year,years,yes,yet,york,young,zealand,zone
0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.165719,0.0,0.0,0.000000
1,0.0,0.092627,0.000000,0.036981,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.028335,0.018144,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.021879,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,0.0,0.035650,0.033640,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
17876,0.0,0.000000,0.000000,0.034302,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.033153,0.0,0.0,0.000000,0.0,0.0,0.000000
17877,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.048228,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.045746,0.0,0.0,0.000000,0.0,0.0,0.000000
17878,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [45]:
text_columns = ['company_profile', 'description', 'requirements', 'benefits']

for i in text_columns:
  df[i] = df_drop[i].fillna("unspecified")

df_drop['title'] = df_drop['title'].fillna("Unspecified")
df_drop['requirements'] = df_drop['requirements'].fillna("Unspecified")
df_drop['benefits'] = df_drop['benefits'].fillna("Unspecified")
df_drop['company_profile'] = df_drop['company_profile'].fillna("Unspecified")
df_drop['description'] = df_drop['description'].fillna("Unspecified")

df_drop['has_title'] = df_drop['title'].apply(lambda x: 0 if x == "unspecified" else 1)
df_drop['has_requirements'] = df_drop['company_profile'].apply(lambda x: 0 if x == "unspecified" else 1)
df_drop['has_comp_profile'] = df_drop['description'].apply(lambda x: 0 if x == "unspecified" else 1)
df_drop['has_description'] = df_drop['requirements'].apply(lambda x: 0 if x == "unspecified" else 1)
df_drop['has_benefits'] = df_drop['benefits'].apply(lambda x: 0 if x == "unspecified" else 1)

In [46]:
df_drop.columns

Index(['job_id', 'title', 'department', 'company_profile', 'description',
       'requirements', 'benefits', 'telecommuting', 'has_company_logo',
       'has_questions', 'employment_type', 'required_experience',
       'required_education', 'industry', 'function', 'fraudulent', 'country',
       'state', 'city', 'min_salary', 'max_salary', 'has_requirements',
       'has_comp_profile', 'has_description', 'has_benefits', 'has_title'],
      dtype='object')

In [49]:
df_drop = pd.concat([df_drop, text_column], axis =1, ignore_index=False)

In [52]:
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Columns: 2362 entries, job_id to zone
dtypes: Sparse[float64, 0](2336), float64(2), int64(10), object(14)
memory usage: 31.3+ MB


In [53]:
# x, y split
X = df_drop.drop(columns= ['title', 'text', 'job_id', 'fraudulent', 'company_profile', 'description', 'requirements', 'benefits'])
y = df_drop['fraudulent']

In [57]:
X.head()

,department,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,country,...,writtenlifting,xml,year,years,yes,yet,york,young,zealand,zone
0,Marketing,0,1,0,Other,Internship,NaN,NaN,Marketing,US,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.165719,0.0,0.0,0.0
1,Success,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,NZ,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,US,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Sales,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,US,...,0.0,0.0,0.028335,0.018144,0.0,0.0,0.000000,0.0,0.0,0.0
4,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,US,...,0.0,0.0,0.000000,0.021879,0.0,0.0,0.000000,0.0,0.0,0.0


In [ ]:
#-------------------end cleaning for text columns

In [ ]:
#-------------one hot encode 



#       'telecommuting', 'has_company_logo',
#       'has_questions', 'employment_type', 'required_experience',
#       'required_education', 'industry', 'function','country',
#       'state', 'city','has_requirements',
#       'has_comp_profile', 'has_description', 'has_benefits', 'has_title'



# scale        'min_salary', 'max_salary'

In [ ]:
# over&undersampling

In [ ]:
# split test and training

In [ ]:
# modelling